# Trabajo Práctico 1 : Reservas de Hotel

### Grupo 01 - Integrantes:
        Cabrera Rodríguez, Mateo 
        Gonzalez Alejo, Camila
        Bocanegra, Eduardo Martín

In [16]:
# Manipulacion de datos y funciones matemáticas
import pandas as pd
import numpy as np

# Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#modelos y métricas
from sklearn import tree
from sklearn.metrics import 
from sklearn.model_selection import train_test_split,  StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [3]:
ds = pd.read_csv('../datasets/hotels_train.csv')

In [29]:
target = 'is_canceled'
features = [col for col in ds.columns if col != target]
print(features, '\n', target)

['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status_date', 'id'] 
 is_canceled


In [30]:
#Separo un set de Evaluacion
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ds[features].values, 
                                                    ds[target].values, 
                                                    test_size=0.2,
                                                    random_state=13,  #para poder reproducir el experimento
                                                    stratify=ds[target].values) #estratificado para mantener proporcion

print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))

for split_name, split in zip(['% Positivos Entrenamiento','% Positivos Prueba'],[y_train,y_test]):
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[1]*100))

# Datos Entrenamiento: 49198
# Datos Prueba: 12300
% Positivos Entrenamiento: 50.118
% Positivos Prueba: 50.114


In [4]:
# encuentro variables categoricas
categorical_columns = [col for col in ds.columns if ds[col].dtype == 'object' or ds[col].dtype == 'string']
print(categorical_columns)

['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status_date', 'id']


In [5]:
ds['children'].fillna(0, inplace=True)
ds['children'] = ds['children'].astype('int64')
ds = ds.drop(ds[ds['children'] > 8].index)
ds = ds.drop(ds[ds['babies'] > 8].index)
ds = ds.drop(ds[ds['adults'] == 0].index)
ds = ds.dropna(subset=['country'])

In [6]:
agents = ds['agent'].unique()
agents = np.delete(agents, 1)
ds['agent'].fillna(0, inplace=True)

companies = ds['company'].unique()
companies = np.delete(companies, 0)
ds['company'].fillna(0, inplace=True)

In [7]:
ds = ds.replace({'meal': {'BB': 0, 'HB': 1, 'FB': 2, 'SC': 3, 'Undefined': 4},# ver que hacer con undefined
                 'arrival_date_month': {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,'November': 11, 'December': 12},
                 'reserved_room_type': {a : n for n, a in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", 'L'])},
                 'assigned_room_type': {a : n for n, a in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K",'L'])},
                 'deposit_type': {'No Deposit': 0, 'Refundable': 1, 'Non Refund': 2},
                 'agent': {agent : 1 for agent in agents},
                 'company': {company : 1 for company in companies}})

In [8]:
print(ds['assigned_room_type'].unique())

[ 3  0  5  4  7  2  6  8  1 10 11]


In [9]:
label_encoder = LabelEncoder()
ds['market_segment'] = label_encoder.fit_transform(ds['market_segment'])
ds['distribution_channel'] = label_encoder.fit_transform(ds['distribution_channel'])
ds['customer_type'] = label_encoder.fit_transform(ds['customer_type'])

In [10]:
categorical_columns = [col for col in ds.columns if ds[col].dtype == 'object' or ds[col].dtype == 'string']
print(categorical_columns)

['hotel', 'country', 'reservation_status_date', 'id']


In [23]:
ds['is_canceled'].value_counts(normalize=True)*100

1    50.117077
0    49.882923
Name: is_canceled, dtype: float64

In [18]:
X = ds.drop(['hotel', 'country', 'reservation_status_date', 'id', 'is_canceled'], axis=1) # eliminar country, reservation_status_date, company cuando se cargue el ds chp1
y = ds['is_canceled']

# PARAMETROS
n = 10
FOLDS = 5
kfold = StratifiedKFold(n_splits=FOLDS)
scorer_fn = make_scorer(sk.metrics.f1_score)

parametros = {'criterion': ['gini', 'entropy'],
                'min_samples_leaf': list(range(1, 10)),
                'min_samples_split': list(range(2, 20)),
                'ccp_alpha': np.linspace(0, 0.005, n),
                'max_depth': list(range(1, 10))
}

modelo = tree.DecisionTreeClassifier()
grid = GridSearchCV(modelo, parametros, scoring=scorer_fn, cv=kfold, n_jobs=-1)

grid.fit(X, y)

print(grid.best_score_)
print(grid.best_params_)


In [19]:
#nos anotamos los mejores parametros para no tener que correr todo de nuevo
best_params = {'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'ccp_alpha': 0.0} #asumidos, despues corremos el grid search 

In [21]:
mejor_arbol = tree.DecisionTreeClassifier(criterion=best_params['criterion'], max_depth=best_params['max_depth'], min_samples_leaf=best_params['min_samples_leaf'], min_samples_split=best_params['min_samples_split'], ccp_alpha=best_params['ccp_alpha'])
mejor_arbol.fit(X, y)

dot_data = StringIO()
export_graphviz(mejor_arbol, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=features,
                class_names=['good','bad'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH